In [1]:
delete this line
i
import pandas as pd
import os 
import h3 as h3
import matplotlib.pyplot as plt
import plotly
import folium
import webbrowser
from folium import Map

SyntaxError: invalid syntax (<ipython-input-1-dc6aeedd7bff>, line 1)

In [ ]:
from IPython.core.display import display, HTML
import json
from pandas.io.json import json_normalize
import numpy as np

import statistics

from geojson.feature import *
# from area import area

import copy
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display
from IPython.utils.text import columnize
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# parl_const = pd.read_csv('~/Downloads/Book3.csv')    # parliamentary constituencies
# df = pd.read_csv("~/Downloads/9999_pc_with_latlongs.csv")   # Food bank locations
df = pd.read_csv('~/Downloads/Book3_copy3.csv')    # parliamentary constituencies
ori = pd.read_csv('~/Downloads/Book3_copy3.csv')    # parliamentary constituencies
print(df.head())

In [ ]:
# Simplify column names
# parl_const.columns = ['lat','lng']
# test = parl_const.sample(frac=0.05, replace=False, random_state=1)
# test.plot(x='lng',y='lat',style='.',alpha=0.2,figsize=(12,12));
# plt.title('UK constituencies');

In [2]:


def counts_by_hexagon(df, resolution):

    '''Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
      Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons'''

    df = df[["lat","lng"]]

    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["lat"], row["lng"], resolution), axis = 1)

    df_aggreg = df
    df_aggreg['no_of_foodbanks'] = ori['no_of_foodbanks']
    df_aggreg['constituency'] = ori['constituency']

    df_aggreg.columns = ["lat", "lng", "hex_id", "no_of_foodbanks", "constituency"]

    df_aggreg["geometry"] =  df_aggreg.hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(x,geo_json=True)]
                                                            }
                                                        )

    return df_aggreg


def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    """
    Produce the GeoJSON for a dataframe that has a geometry column in geojson 
    format already, along with the columns hex_id and value

    Ex counts_by_hexagon(data)
    """    
    list_features = []

    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"no_of_foodbanks" : row["no_of_foodbanks"]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    geojson_result = json.dumps(feat_collection)

    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)

    return geojson_result

def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
               kind = "linear"):

    """
    Creates choropleth maps given the aggregated data.
    """    
    #colormap
    min_value = df_aggreg["no_of_foodbanks"].min()
    max_value = df_aggreg["no_of_foodbanks"].max()
    m = round ((min_value + max_value ) / 2 , 0)

    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'hex_id'])

    if initial_map is None:
        initial_map = Map(location= [51.53, 0.11], zoom_start=7, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )


    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)


    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)

    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind

    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['no_of_foodbanks']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)
    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)



    return initial_map

def plot_scatter(df, metric_col, x='lng', y='lat', marker='.', alpha=1, figsize=(16,12), colormap='viridis'):
    """
    Scatter plot function for h3 indexed objects
    """    
    df.plot.scatter(x=x, y=y, c=metric_col, title=metric_col
                    , edgecolors='none', colormap=colormap, marker=marker, alpha=alpha, figsize=figsize);
    plt.xticks([], []); plt.yticks([], [])


In [3]:
# Counts how many points are within the hex
myres = 6
df_aggreg = counts_by_hexagon(df = df, resolution = myres)      #Use 4 to tile the whole of uk in an aesthetically pleasing way
df_aggreg.sort_values(by = "no_of_foodbanks", ascending = False, inplace = True)

# Creates a map using Folium
hexmap = choropleth_map(df_aggreg = df_aggreg, with_legend = True)
# hexmap.save('downloads')
from folium import plugins
plugins.Fullscreen(position='topright').add_to(hexmap)
draw = plugins.Draw(export=True)
hexmap


NameError: name 'df' is not defined

In [ ]:
df_aggreg

In [ ]:
# h3.h3_to_geo('85194ad3fffffff')

In [ ]:
def show_map(themap, file_name):
    themap.save(file_name)
    wb = webbrowser.open('file://' + os.path.realpath(file_name), new=2)     #opens in new tab. using a new of 1 opens in a new window

In [ ]:
# show_map(hexmap, "h3_level:_{}_.html".format(myres))

In [ ]:
column2_as_series = set(df_aggreg['hex_id'])
column2_as_series

In [ ]:
len(column2_as_series)

In [ ]:
# Work out mean number of foodbanks in each hexagon

df_aggreg["no_of_foodbanks"].mean()

In [ ]:
# Calculate the standard deviation for the number of foodbanks

df_aggreg["no_of_foodbanks"].std()